Set up environment and load in data

In [ ]:
library(tidyverse)
library(here)
theme_set(theme_bw())
helpers_path = paste0(here(),'/analysis/helpers/')
# source(paste0(helpers_path, '01_clean_behavioral_data.R'))
# rm(data_bc_clean)
fig_out_path = paste0(here(), '/outputs/fig/')
cpu_eaters_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/'

# Grid search results

Fit grid search for one subject (601).  

Value was normalized for each job (i.e. each day for each stimulus type) to range between -1 and 1.

Grid from `save_ddm_grid.R`

```
d = seq(0.01, .1, .01)
sigma = seq(0.01, .1, .01)
ndt = seq(200, 400, 100)
bias = seq(-.1, .1, .1)
barrierDecay = c(0, .001, .01, .02)
```

Not parallelized over grid. Parallelized only over day and type. So there were 22 jobs (11 days x 2 types). Each job computed the likelihood of observed data for 3600 parameter combinations (10 x 10 x 3 x 3 x 4) unless anything errored out/job was cancelled for taking too long.  

Boundary separation fixed at 2 (but allowed to decay if `barrierDecay` != 0).  

Using state-space approach with an approximate step size of 0.01.  

Note that there won't be any posterior distributions in this approach.  

## Single job output

Output for a single job looks like:   

In [ ]:
grid_search_outputs = list.files(paste0(cpu_eaters_path, 'ddm/grid_search_out'))

tmp = read.csv(paste0(cpu_eaters_path, 'ddm/grid_search_out/', grid_search_outputs[1]))

tmp

How to examine a single job's output/parameter estimates for one day and stim type

The simplest thing is to just get the parameter combination with the highest likelihood/smallest nll.  

In [ ]:
tmp %>%
  arrange(nll) %>%
  slice(1)

But is this parameter combination obviously better than the others? What does the distribution of the likelihood look like?

The whole distribution shows some very large nll's, i.e. some very bad fits to data.

In [ ]:
tmp %>% 
  ggplot(aes(nll)) +
  geom_histogram(alpha = .5, bins = 30)

Half of the data has a nll larger than 3809.

In [ ]:
summary(tmp$nll)

So the distribution of likelihoods for the better fitting half of the parameter combination looks like:

In [ ]:
tmp %>% 
  filter(nll < 3809) %>%
  ggplot(aes(nll)) +
  geom_histogram(alpha = .5, bins = 30)

Are there clearly terrible parameter values for the combinations that are in the better fitting half?

In [ ]:
tmp %>%
  filter(nll < 3809) %>%
  select(-subnum, -day, -type, -model) %>%
  gather(key, value, -nll) %>%
  ggplot(aes(as.factor(value), nll))+
  geom_boxplot()+
  facet_wrap(~key, scales = 'free')+
  labs(x = "")

## All jobs' outputs

Things to check for each job:
- Distribution of likelihoods
- Bad areas of parameter space
- Number of failed jobs

In [ ]:
all_pars = tibble()

for(i in grid_search_outputs){
  cur_out = read.csv(paste0(cpu_eaters_path, 'ddm/grid_search_out/', i))
  all_pars = rbind(all_pars, cur_out)
}

rm(cur_out, i)

There are some failed jobs for HT day 1 and day 2. All others seem to have completed likelihood calculation for all 3600 parameter combinations.

In [ ]:
all_pars %>%
  group_by(day, type) %>%
  summarise(count = n(), .groups = "keep") %>%
  arrange(count)

### NLL distributions

In [ ]:
all_pars %>%
  group_by(day, type) %>%
  ggplot(aes(nll)) +
  geom_histogram(alpha = .5, bins = 30) +
  facet_grid(type ~ day) +
  labs(title = "Full NLL distributions")

In [ ]:
all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  ggplot(aes(nll)) +
  geom_histogram(alpha = .5, bins = 30) +
  facet_grid(type ~ day, scales = "free") +
  labs(title = "Lower 50% NLL distributions")

In [ ]:
p = all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  select(-subnum, -model) %>%
  gather(key, value, -nll, -day, -type) %>%
  filter(key == "d") %>%
  ggplot(aes(as.factor(value), nll))+
  geom_violin()+
  facet_grid(type ~ day, scales = 'free')+
  labs(x = "d", title = "NLL distribution for each d collapsed over other parameters (for 50% best NLLs)") +
  theme(panel.grid.minor = element_blank(),
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank())

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_NLLforD.jpg'), p, height = 4.5, width=12, units="in")
p

In [ ]:
p = all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  select(-subnum, -model) %>%
  gather(key, value, -nll, -day, -type) %>%
  filter(key == "sigma") %>%
         # value > .05) %>%
  ggplot(aes(as.factor(value), nll))+
  geom_violin()+
  facet_grid(type ~ day, scales = 'free')+
  labs(x = "sigma", title = "NLL distribution for each sigma collapsed over other parameters (for 50% best NLLs)")+
  theme(panel.grid.minor = element_blank(),
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank())

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_NLLforSigma.jpg'), p, height = 4.5, width=12, units="in")
p

In [ ]:
p = all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  select(-subnum, -model) %>%
  gather(key, value, -nll, -day, -type) %>%
  filter(key == "nonDecisionTime") %>%
  ggplot(aes(as.factor(value), nll))+
  geom_violin()+
  facet_grid(type ~ day, scales = 'free')+
  labs(x = "NDT", title = "NLL distribution for each ndt collapsed over other parameters (for 50% best NLLs)")+
  theme(panel.grid.minor = element_blank(),
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank())

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_NLLforNDT.jpg'), p, height = 4.5, width=12, units="in")
p

In [ ]:
p = all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  select(-subnum, -model) %>%
  gather(key, value, -nll, -day, -type) %>%
  filter(key == "bias") %>%
  ggplot(aes(as.factor(value), nll))+
  geom_violin()+
  facet_grid(type ~ day, scales = 'free')+
  labs(x = "Bias", title = "NLL distribution for each bias collapsed over other parameters (for 50% best NLLs)") +
  theme(panel.grid.minor = element_blank(),
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank())

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_NLLforbias.jpg'), p, height = 4.5, width=12, units="in")
p

In [ ]:
p = all_pars %>%
  group_by(day, type) %>%
  mutate(median_nll = median(nll)) %>%
  filter(nll > median_nll) %>%
  select(-subnum, -model) %>%
  gather(key, value, -nll, -day, -type) %>%
  filter(key == "barrierDecay") %>%
  ggplot(aes(as.factor(value), nll))+
  geom_violin()+
  facet_grid(type ~ day, scales = 'free')+
  labs(x = "Barrier Decay", title = "NLL distribution for each barrierDecay collapsed over other parameters (for 50% best NLLs)")+
  theme(panel.grid.minor = element_blank(),
        axis.ticks.x = element_blank(),
        axis.text.x = element_blank())

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_NLLforBarrierDecay.jpg'), p, height = 4.5, width=12, units="in")
p

### Best fitting parameters

Differences in best fitting parameters across stim types and days

In [ ]:
best_pars = tibble()

for(i in grid_search_outputs){
  cur_out = read.csv(paste0(cpu_eaters_path, 'ddm/grid_search_out/', i))
  cur_best = cur_out %>%
    arrange(nll) %>%
    slice(1)
  best_pars = rbind(best_pars, cur_best)
}

rm(cur_out, cur_best, i)

Notable things:

- Best fitting parameters have barrierDecay>0
- Barrier decay is either the same for the two conditions or higher for HT
- Barrier decay is higher on later days

- Bias is mostly 0 (--> consider fixing it?)
- When bias is not 0, it's positive for HT (yes) and negative for RE (no)

- Drift rates increase across days
- Drift rates are higher for HT

- Non decision times are more often higher for HT (unexpected)
- Generally constant across days (good)

- Noise is higher for HT half the time and higher for RE for the other half
- No clear pattern of increase or decrease over days

In [ ]:
p = best_pars %>%
  select(-subnum, -model, -nll) %>%
  gather(key, value, -day, -type) %>%
  ggplot(aes(as.factor(day), value, color = type))+
  geom_point(alpha = .5)+
  facet_wrap(~key, scales = "free")+
  scale_color_brewer(palette = "Dark2")+
  labs(x = "Day", y = "", color = "")+
  theme(legend.position = "bottom")

# ggsave(file=paste0(fig_out_path, 'sub601_gridSearch_best pars.jpg'), p, height = 4.5, width=9, units="in")
p

# Optim out results

Fit for 3 subjects (601, 609 and 611) for both conditions and all days.

Value was normalized for each job (i.e. each day for each stimulus type) to range between -1 and 1.

Ideally, the algorithm should converge on the same/similar parameters for each job (fit for subject, day, type).

But since I'm not sure how robust it is I had planned to have it start from each of the parameter combinations used for the grid search. This would have meant trying 3600 different starting values and then checking if the algorithm converges (after max 500 iterations) to the same place.  

It still doesn't give full posteriors but at least we would have had a little more support for the converged value.  
I did not parallelize across starting values. So this ended up taking forever and I only have a few outputs for each job (subject, day, type) instead of the full 3600.

In [ ]:
optim_outputs = list.files(paste0(cpu_eaters_path, 'ddm/optim_out'))

tmp = read.csv(paste0(cpu_eaters_path, 'ddm/optim_out/', optim_outputs[1]))

tmp

## All jobs' outputs

In [ ]:
all_optim = tibble()

for(i in optim_outputs){
  cur_out = read.csv(paste0(cpu_eaters_path, 'ddm/optim_out/', i))
  all_optim = rbind(all_optim, cur_out)
}

rm(cur_out, i)


How many converged iterations per job? Max = 12, min 4.

In [ ]:
all_optim %>%
  group_by(day, subnum, type) %>%
  summarise(.groups = "keep",
            max_iter = max(start_num)) %>%
  arrange(-max_iter)

Excluding non-sensical values for each paramter is there a converged parameter combination for each job (subject, day, type combination)? Yes.

In [ ]:
all_optim %>%
  filter(barrierDecay > 0 & barrierDecay < 1) %>%
  filter(bias < 1) %>%
  filter(d > 0) %>%
  filter(sigma > 0) %>%
  filter(nonDecisionTime > 0) %>%
  group_by(day, subnum, type) %>%
  summarise(.groups = "keep",
            max_iter = max(start_num)) %>%
  arrange(-max_iter)

In [ ]:
all_optim_filtered = all_optim %>%  
  filter(barrierDecay > 0 & barrierDecay < 1) %>%
  filter(bias < 1) %>%
  filter(d > 0) %>%
  filter(sigma > 0) %>%
  filter(nonDecisionTime > 0)

How far are the converged parameters from their starting value?   

There should have been 10 x 10 x 3 x 3 x 4 starting values.  

```
d = seq(0.01, .1, .01)
sigma = seq(0.01, .1, .01)
ndt = seq(200, 400, 100)
bias = seq(-.1, .1, .1)
barrierDecay = c(0, .001, .01, .02)
```

But instead there is only 1 barrierDecay and bias starting value, 3 drift rate starting values, 2 ndt starting values and 3 noise level starting values.

In [ ]:
unique(all_optim$start_barrierDecay)
unique(all_optim$start_bias)
unique(all_optim$start_d)
unique(all_optim$start_nonDecisionTime)
unique(all_optim$start_sigma)

Filtering converged jobs with non-sensical values does not remove a specific starting value for any of the parameters.

In [ ]:
unique(all_optim_filtered$start_barrierDecay)
unique(all_optim_filtered$start_bias)
unique(all_optim_filtered$start_d)
unique(all_optim_filtered$start_nonDecisionTime)
unique(all_optim_filtered$start_sigma)

### Distance from start value

To check if things have moved from their starting value I plot the histograms for the difference between the converged value and the starting value for each parameter across jobs.  

This only looks at converged jobs that have sensible values for all parameters.

In [ ]:
all_optim_filtered %>%
  mutate(diff_barrierDecay = start_barrierDecay - barrierDecay,
         diff_bias = start_bias - bias,
         diff_d = start_d - d,
         diff_nonDecisionTime = start_nonDecisionTime - nonDecisionTime,
         diff_sigma = start_sigma - sigma) %>%
  select(diff_d, diff_sigma, diff_nonDecisionTime, diff_bias, diff_barrierDecay) %>%
  gather(key, value) %>%
  ggplot(aes(value))+
  geom_histogram(bins = 20, alpha = .5) + 
  facet_wrap(~key, scales = "free")

How many iterations did it take to converge per starting value (collapsing across subjects, days, types)

In [ ]:
all_optim_filtered %>%
  ggplot(aes(optim_iters)) + 
  geom_histogram(alpha = .5, bins = 20)

### Variance in converged estimates

How different are the converged estimates from each other for each job (subject, day, type combination) per starting value?

They can be quite different from each other especially when they do move further away from their starting values. The estimates for barrrierDecay and ndt differ less from each other depending on the converged value but they also don't move as far away from their starting values either.

In [ ]:
all_optim_filtered %>%
  select(subnum, day, type, bias, barrierDecay, d, sigma, nonDecisionTime) %>%
  gather(key, value, -subnum, -day, -type) %>%
  group_by(subnum, day, type, key) %>%
  mutate(key = ifelse(key == "nonDecisionTime", "NDT", ifelse(key == "barrierDecay", "decay", key))) %>%
  ggplot(aes(as.factor(day), value, color = type))+
  geom_point(position = position_dodge(width=.5))+
  facet_grid(key~subnum, scales = "free")+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")+
  labs(color = "", y = "Converged estimate", x = "Day")

In [ ]:
all_optim_filtered %>%
  mutate(diff_barrierDecay = start_barrierDecay - barrierDecay,
         diff_bias = start_bias - bias,
         diff_d = start_d - d,
         diff_nonDecisionTime = start_nonDecisionTime - nonDecisionTime,
         diff_sigma = start_sigma - sigma) %>%
  select(subnum, day, type, diff_d, diff_sigma, diff_nonDecisionTime, diff_bias, diff_barrierDecay) %>%
  gather(key, value, -subnum, -day, -type) %>%
  group_by(subnum, day, type, key) %>%
  ggplot(aes(as.factor(day), value, color = type))+
  geom_point(position = position_dodge(width=.5))+
  geom_hline(aes(yintercept = 0)) +
  facet_grid(key~subnum, scales = "free")+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")+
  labs(color = "", y = "Difference between starting and converged value", x = "Day")

Are these converged estimates similiar in fit (i.e. nll)?

In [ ]:
all_optim_filtered %>%
  ggplot(aes(as.factor(day), nll, fill = type))+
  geom_boxplot(position = position_dodge(width = .25))+
  facet_grid(type~subnum, scales = "free")+
  labs(x = "Day", fill = "")+
  theme(legend.position = "none")+
  scale_fill_brewer(palette = "Dark2")

# Single subject optim and grid search comparison

How close are the best estimates from optim to the single subject (601) grid search best parameters?

In [ ]:
sub_grid_best = best_pars %>%
  select(-model) %>%
  mutate(fit = "grid")


In [ ]:
sub_optim_best = all_optim_filtered %>%
  filter(subnum == 601) %>%
  group_by(subnum, day, type) %>%
  filter(nll == min(nll)) %>%
  select(barrierDecay, bias, d, nonDecisionTime, sigma, nll, subnum, day, type) %>%
  mutate(fit = "optim")

### NLL in optim vs grid search

Compare likelihoods of best fitting parameters from two approaches. 

Notable things:

- Model fits RE better than HT for all days.  
- Between the two fitting schemes the likelihood of the best fitting parameter combinations is almost identical.

In [ ]:
sub_grid_best %>%
  select(nll, subnum, day, type, fit) %>%
  rbind(sub_optim_best %>%
          select(nll, subnum, day, type, fit)) %>%
  spread(fit, nll) %>%
  ggplot(aes(grid, optim, color = type))+
  geom_point()+
  geom_abline(aes(slope = 1, intercept = 0))+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")+
  labs(color = "", y = "NLL from optim", x = "NLL from grid search")

### Parameter estimates for optim vs grid search

For these almost identical nll's are the converged parameters similar? Yes and no. The exact estimates are not identifical but patterns, especially comparing the two conditions are captured.

- The best barrierDecays for RE are often lower than those for HT in both methods.  
- When grid search converges on a non-zero bias the sign of the bias always matches with optim too. When grid search converges on 0 the biggest difference is still smaller than the other best grid search estimates.  
- ndt's are where the largest differences are. Optim estimates much faster ndt's compared to grid search.  
- Drift rates are often larger when using grid search compared to optim. This is possibly somewhat due to the larger ndt's with grid search.  
- The optim sigma estimates range between the two most frequently converged values with grid search ([0.04 - 0.05]).  

In [ ]:
sub_grid_best %>%
  select(-nll) %>%
  gather(key, value, -subnum, -day, -type, -fit) %>%
  rbind(sub_optim_best %>%
          select(-nll) %>%
          gather(key, value, -subnum, -day, -type, -fit)) %>%
  spread(fit, value) %>%
  ggplot(aes(grid, optim, color = type))+
  geom_point(alpha = .5)+
  geom_abline(aes(intercept = 0, slope = 1))+
  theme(legend.position = "bottom")+
  scale_color_brewer(palette = "Dark2")+
  labs(color = "", y = "Estimate from optim", x = "Estimate grid search")+
  facet_wrap(~key, scales = "free")

### Simulated data checks

How much do the exact values of the point estimates matter?  

How different does data generated by the point estimates recovered by the two different methods look from each other AND from true data?  

Source simulation functions.  

In [ ]:
# Source model definition
source(paste0(helpers_path, '/ddm/yn_ddm.R'))

# Source simulating functions
source(paste0(helpers_path, '/ddm/sim_yn_ddm.R'))

# Create list with the simulator function
sim_trial_list = list("model1" = sim_trial)

# Get data for subject
source(paste0(helpers_path, '01_clean_behavioral_data.R'))
rm(data_bc_clean)

normMax = 1
normMin = -1

data_yn_clean = data_yn_clean %>%
  filter(reference != -99) %>%
  filter(rt > .3 & rt < 5) %>% # discard very long and short RT trials
  group_by(subnum, day, type) %>%
  mutate(possiblePayoff_dmn = possiblePayoff - mean(possiblePayoff)) %>%
  mutate(rawVDiff = possiblePayoff - reference,
         normVDiff =  (normMax - normMin) / (max(rawVDiff) - min(rawVDiff)) * (rawVDiff - max(rawVDiff)) + (normMax) )

rm(normMax, normMin)

sub_data = data_yn_clean %>% filter(subnum == 601)

Simulate data using best estimates from grid search

In [ ]:
sim_grid_best = tibble()

for(i in 1:nrow(sub_grid_best)){
  cur_row = sub_grid_best[i,]
  cur_day = cur_row$day
  cur_type = ifelse(cur_row$type == "HT", 1, 0)
  cur_stims = sub_data %>% filter(day == cur_day & type == cur_type)
  cur_sim = sim_task(stimuli = cur_stims, model_name = "model1", sim_trial_list_ = sim_trial_list, 
                     d = cur_row$d, sigma = cur_row$sigma, nonDecisionTime = cur_row$nonDecisionTime, 
                     bias = cur_row$bias, barrierDecay = cur_row$barrierDecay, 
                     maxIter = 300)
  cur_sim$type = cur_type
  cur_sim$day = cur_day
  cur_sim$true_yesChosen = cur_stims$yesChosen
  cur_sim$true_rt = cur_stims$rt
  sim_grid_best = rbind(sim_grid_best, cur_sim)
  
}

rm(cur_row, cur_sim, cur_stims, i)

In [ ]:
sim_grid_best

Simulate data using best parameters from optim

In [ ]:
sim_optim_best = tibble()

for(i in 1:nrow(sub_optim_best)){
  cur_row = sub_optim_best[i,]
  cur_day = cur_row$day
  cur_type = ifelse(cur_row$type == "HT", 1, 0)
  cur_stims = sub_data %>% filter(day == cur_day & type == cur_type)
  cur_sim = sim_task(stimuli = cur_stims, model_name = "model1", sim_trial_list_ = sim_trial_list, 
                     d = cur_row$d, sigma = cur_row$sigma, nonDecisionTime = cur_row$nonDecisionTime, 
                     bias = cur_row$bias, barrierDecay = cur_row$barrierDecay, 
                     maxIter = 300)
  cur_sim$type = cur_type
  cur_sim$day = cur_day
  cur_sim$true_yesChosen = cur_stims$yesChosen
  cur_sim$true_rt = cur_stims$rt
  sim_optim_best = rbind(sim_optim_best, cur_sim)
  
}

rm(cur_row, cur_sim, cur_stims, i)

In [ ]:
sim_optim_best

#### Plot simulated data

Whether the true and simulated data are plotted in the same or different layers doesn't matter BUT make sure to use the same geom for both true and simulated data! Otherwise you'll plot kernel density estimates (smoothed continuous "estimates")  over rescaled histograms bin values and they won't be normalized by the same process so would not be comparable!

Useful threads:  

https://stackoverflow.com/questions/46734555/ggplot2-histogram-why-do-y-density-and-stat-density-differ

https://stackoverflow.com/questions/65631194/scale-density-curve-made-with-geom-density-to-similar-height-of-geom-histogram


In [ ]:
p = sim_optim_best %>%
  select(true_rt, true_yesChosen, reactionTime, choice, type, day) %>%
  mutate(optim_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime),
         true_rt = ifelse(true_yesChosen == 0, (-1)*true_rt, true_rt)) %>%
  select(-true_yesChosen, -reactionTime, -choice) %>%
  gather(key, value, -type, -day) %>%
  rbind(sim_grid_best %>%
          select(reactionTime, choice, type, day) %>%
          mutate(grid_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime)) %>%
          select(-reactionTime, -choice) %>%
          gather(key, value, -type, -day)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1))),
         type = ifelse(type == 1, "HT", "RE"),
         key = factor(key, levels = c("true_rt", "grid_rt", "optim_rt"))) %>%
  ggplot(aes(value, color = key, fill = key))+
  geom_histogram(aes(y=..density..), bins = 30, alpha = .5, position = "identity")+
  facet_grid(type ~ day)+
  scale_color_manual(values = c("gray", "#7570b3", "#e7298a"))+
  scale_fill_manual(values = c("gray", NA, NA))+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.position = "bottom")+
  labs(y = "", x = "", fill = "", color = "")+
  ylim(0, 1.5)

p

In [ ]:
p2 = sim_optim_best %>%
  select(true_rt, true_yesChosen, reactionTime, choice, type, day) %>%
  mutate(optim_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime),
         true_rt = ifelse(true_yesChosen == 0, (-1)*true_rt, true_rt)) %>%
  select(-true_yesChosen, -reactionTime, -choice) %>%
  gather(key, value, -type, -day) %>%
  rbind(sim_grid_best %>%
          select(reactionTime, choice, type, day) %>%
          mutate(grid_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime)) %>%
          select(-reactionTime, -choice) %>%
          gather(key, value, -type, -day)) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1))),
         type = ifelse(type == 1, "HT", "RE"),
         key = factor(key, levels = c("true_rt", "grid_rt", "optim_rt"))) %>%
  ggplot(aes(value, color = key, fill = key))+
  # geom_step(aes(y=..density..), stat="bin", binwidth=0.2)+
  geom_step(aes(y=..density..), stat="bin", bins = 30, linewidth = .75)+
  facet_grid(type ~ day)+
  scale_color_manual(values = c("gray", "#7570b3", "#e7298a"))+
  scale_fill_manual(values = c("gray", NA, NA))+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.position = "bottom")+
  labs(y = "", x = "", fill = "", color = "")+
ylim(0, 1.5)

p2

##### KL divergence

To compute e.g. KL divergence between true and simulated RT distributions you could do something like getting the number of values in each RT bin in the above plot using `ggplot_build(p)`

In [ ]:
pg = ggplot_build(p)

In [ ]:
prob_data = pg$data[[1]] %>%
  select(count, colour, PANEL) %>%
  group_by(colour, PANEL) %>%
  mutate(p_count = count/sum(count),
         group = ifelse(colour == "gray", "true_rt", ifelse(colour == "#7570b3", "grid_rt", "optim_rt"))) %>%
  # summarise(sum_p_count = sum(p_count), .groups = "keep") # Check to make sure p's add up to 1 for each case
  left_join(pg$layout$layout %>% select(PANEL, type, day), by = "PANEL") 

f_labels = tibble()

for(cur_panel in unique(prob_data$PANEL)){
  panel_data = prob_data %>% filter(PANEL == cur_panel)
  true_probs = panel_data %>% filter(group == "true_rt")
  true_probs = true_probs$p_count
  
  grid_probs = panel_data %>% filter(group == "grid_rt")
  grid_probs = grid_probs$p_count
  x = rbind(true_probs, grid_probs)
  y1 = philentropy::KL(x, unit='log2')
  
  optim_probs = panel_data %>% filter(group == "optim_rt")
  optim_probs = optim_probs$p_count
  x = rbind(true_probs, optim_probs)
  y2 = philentropy::KL(x, unit='log2')
  
  cur_label = paste0("KL(tg) = ", format(round(y1, 3), nsmall = 3), " KL(to) = ",  format(round(y2, 3), nsmall = 3))
  
  cur_row = tibble(type = unique(panel_data$type), day = unique(panel_data$day),
                   label = cur_label, kl_tg = y1, kl_to = y2)
  f_labels = rbind(f_labels, cur_row)
}

f_labels$key = "true_rt"
f_labels$label = stringr::str_wrap(f_labels$label, 15)

rm(panel_data, true_probs, grid_probs, optim_probs, cur_label, cur_row, x, y1, y2)

In [ ]:
p +
  geom_text(x = 0, y = 1.2, aes(label = label), data = f_labels, color = "black", size = 2.5)+
  labs(title = "True vs simulated RTs using best parameters from different fitting methods")+
  theme(axis.ticks.y = element_blank(),
        axis.text.y = element_blank())+
  lemon::scale_x_symmetric()


In [ ]:
p2+
  geom_text(x = 0, y = 1.2, aes(label = label), data = f_labels, color = "black", size = 2.5)+
  labs(title = "True vs simulated RTs using best parameters from different fitting methods")+
  theme(axis.ticks.y = element_blank(),
        axis.text.y = element_blank())+
  lemon::scale_x_symmetric()

# ggsave(file=paste0(fig_out_path, 'sub601_barrierDecay_simData.jpg'), height = 6, width=12, units="in")

### Comparison to model without bounds

In [ ]:
stim_types = c("HT", "RE")
subnums = c(601)

in_path = '/Users/zeynepenkavi/CpuEaters/NovelVsRepeated/behavior/analysis/helpers/cluster_scripts/hddm/jags_out'

yn_ddm_summary = data.frame()

for(i in 1:length(stim_types)){
  for(j in 1:length(subnums)){
    
    cur_stim_type = stim_types[i]
    cur_sub = subnums[j]
    # summary_YN_HDDM_FIT_RE_sub-611.csv
    
    fn = file.path(in_path, paste0('summary_YN_HDDM_FIT_', cur_stim_type,'_sub-', cur_sub,'.csv'))
    
    if(file.exists(fn)){
      cur_summary = read.csv(fn)
      cur_summary = cur_summary %>%
        rename(par = X) %>%
        mutate(stim_type = cur_stim_type,
               subnum = cur_sub)
      
      yn_ddm_summary = rbind(yn_ddm_summary, cur_summary)
    } else {
      next
    }
  }
}

In [ ]:
yn_ddm_summary = yn_ddm_summary %>%
  select(par, Lower95, Upper95, Mean, subnum, stim_type ) %>%
  separate(par, c("a", "b"), sep = '\\.', remove = F, fill = "right") %>%
  separate(a, c("c", "d"), sep = "\\[", remove = F, fill = "right") %>%
  select(-a) %>%
  mutate(d = paste0('p[',d)) %>%
  mutate(b = ifelse(is.na(b), d, b)) %>%
  select(-d, -par) %>%
  rename(par_name = c, par_group = b) %>%
  mutate(par_group = gsub("p\\[", "", par_group),
         par_group = gsub("\\]", "", par_group)) %>%
  filter(par_name != "deviance") %>%
  filter(par_group != "kappa" & par_group != "pr") %>%
  filter(par_group != "mu") %>% #Not showing group estimates in this plot
  rename(day = par_group) %>%
  mutate(par_name = ifelse(par_name == "theta", "ndt", ifelse(par_name == "b", "d", ifelse(par_name == "alpha", "sigma", par_name)))) %>%
  mutate(par_name = factor(par_name, levels = c("d", "sigma", "bias", "ndt")),
         day = as.numeric(day))

Simulation helper functions

In [ ]:
sim_trial = function(trial_vdiff, sampled_sigma, sampled_d, sampled_ndt, sampled_bias){
  
  # Multiplying boundary separation and drift rate my sigma to account for that parameter
  trial_drift = sampled_sigma * (sampled_d * trial_vdiff)
  
  if(abs(trial_drift) < 10){
    cur_pred = RWiener::rwiener(1, 2*sampled_sigma, sampled_ndt, sampled_bias, trial_drift)
  } else {
    print("d too large. Sampling RT.")
    pq = sampled_ndt + abs(rnorm(1, mean = 0, sd = 0.01))
    pr = ifelse(trial_drift>10, factor("upper", levels = c("upper", "lower")), factor("lower", levels = c("upper", "lower")))
    cur_pred = data.frame(q = pq, resp = pr)
  }
  return(cur_pred)
  
}

sim_subj_cond_day_once = function(cur_stims, sampled_sigma, sampled_d, sampled_ndt, sampled_bias){
  
  pred_data = cur_stims
  
  normMax = 1
  normMin = -1
  # Already filtered for sub, day and stim type so don't need to group to make sure vdiff ranges between [-1,1]
  pred_data = pred_data %>%
    mutate(rawVDiff = possiblePayoff - reference,
           normVDiff =  (normMax - normMin) / (max(rawVDiff) - min(rawVDiff)) * (rawVDiff - max(rawVDiff)) + (normMax) )
  
  pred_data$sampled_sigma = sampled_sigma
  pred_data$sampled_d = sampled_d
  pred_data$sampled_ndt = sampled_ndt
  pred_data$sampled_bias = sampled_bias
  pred_data$pred_rt = NA
  pred_data$pred_yesChosen = NA
  
  for(i in 1:nrow(cur_stims)){
    
    trial_vdiff = pred_data$normVDiff[i]
    cur_pred = sim_trial(trial_vdiff, sampled_sigma, sampled_d, sampled_ndt, sampled_bias)
    
    pred_data$pred_rt[i] = cur_pred$q
    pred_data$pred_yesChosen[i] = cur_pred$resp
    
  }
  
  return(pred_data)
}

In [ ]:
conds = c('HT', 'RE')
subs = c('601')
days = seq(1, 11, 1)

for(i in 1:length(conds)){
  cur_type = conds[i]
  
  for (j in 1:length(subs)){
    cur_sub = subs[j]
    
    for (k in 1:length(days)){
      cur_day = days[k]
      
      cur_stims = data_yn_clean %>%
        mutate(type_chr = ifelse(type == 1, "HT", "RE")) %>%
        filter(subnum == cur_sub & day == cur_day & type_chr == cur_type) %>%
        select(subnum, day, type, possiblePayoff, reference, yesChosen, rt)
      
      cur_pars = yn_ddm_summary %>%
        filter(subnum == cur_sub & day == cur_day & stim_type == cur_type) %>%
        select(par_name, Mean) %>%
        spread(par_name, Mean)
      
      cur_pred = sim_subj_cond_day_once(cur_stims, cur_pars$sigma, cur_pars$d, cur_pars$ndt, cur_pars$bias)
      
      if(!exists("sim_yn_map")){
        sim_yn_map = cur_pred
      } else{
        sim_yn_map = rbind(sim_yn_map, cur_pred)
      }
    }
  }
}

In [ ]:
rm(cur_pars, cur_pred, cur_stims, cur_summary, cur_day, cur_stim_type, cur_sub, cur_type)
rm(i, j, k)

In [ ]:
sim_yn_map = sim_yn_map %>%
  mutate(pred_yesChosen = ifelse(pred_yesChosen == 1, pred_yesChosen, 0),
         val_diff = possiblePayoff - reference,
         day_std = (day - mean(day))/sd(day)) %>%
  group_by(subnum) %>%
  mutate(val_diff_std = (val_diff - mean(val_diff))/sd(val_diff),
         abs_val_diff = abs(val_diff),
         abs_val_diff_std = (abs_val_diff - mean(abs_val_diff))/sd(abs_val_diff))

sim_yn_map = sim_yn_map %>%
  mutate(val_diff_bin = round(val_diff/50),
         val_diff_bin_str = paste0(val_diff_bin*50-25,":",val_diff_bin*50+25),
         val_diff_bin_str = factor(val_diff_bin_str, levels = c("-225:-175", "-175:-125", "-125:-75", "-75:-25", "-25:25", "25:75", "75:125", "125:175", "175:225")),
         abs_val_diff_bin_str =  paste0(abs(val_diff_bin)*50-25,":",abs(val_diff_bin)*50+25),
         abs_val_diff_bin_str = ifelse(abs_val_diff_bin_str ==  "-25:25", "0:25", abs_val_diff_bin_str),
         abs_val_diff_bin_str = factor(abs_val_diff_bin_str, levels = c("0:25", "25:75", "75:125", "125:175", "175:225", "225:275")))

Check for effect of thresholding the drift rate: Not a big difference.

In [ ]:
sim_yn_map$drift_threshold = NA
sim_yn_map$drift_threshold[1: (nrow(sim_yn_map)/2) ] = 1
sim_yn_map$drift_threshold[ ((nrow(sim_yn_map)/2) + 1) : (nrow(sim_yn_map))] = 0
table(sim_yn_map$drift_threshold)
sim_yn_map

In [ ]:
sim_yn_map %>%
  mutate(type_chr = ifelse(type == 1, "HT", "RE")) %>%
  filter(rt < 3 & pred_rt < 3) %>%
  mutate(rt = ifelse(yesChosen == 1, rt, rt*(-1)),
         pred_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1)), 
         day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1)))) %>%
  ggplot()+
  geom_density(aes(x = pred_rt, linetype = as.factor(drift_threshold)), position = "identity", alpha = .5) +
  geom_histogram(aes(x = rt, y = ..density.., fill = type_chr), bins = 30,  position = "identity", alpha = .5, color = NA)+
  facet_grid(type_chr ~ day)+
  theme(panel.grid = element_blank(),
        axis.text = element_blank(),
        axis.ticks = element_blank(),
        legend.position = "bottom")+
  geom_vline(aes(xintercept = 0), color = "gray") +
  labs(y = "", x = "")+
  scale_fill_brewer(palette = "Dark2")+
  guides(fill = "none")

In [ ]:
sim_yn_map %>%
  filter(drift_threshold == 1) %>%
  ungroup() %>%
  mutate(hddm_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1))) %>%
  select(type, day, hddm_rt) %>%
  gather(key, value, -type, -day)

Compare to optim and grid search simulations

In [ ]:
sub_all_sim_data_long = sim_optim_best %>%
  select(true_rt, true_yesChosen, reactionTime, choice, type, day) %>%
  mutate(optim_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime),
         true_rt = ifelse(true_yesChosen == 0, (-1)*true_rt, true_rt)) %>%
  select(-true_yesChosen, -reactionTime, -choice) %>%
  gather(key, value, -type, -day) %>%
  
  rbind(sim_grid_best %>%
          select(reactionTime, choice, type, day) %>%
          mutate(grid_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime)) %>%
          select(-reactionTime, -choice) %>%
          gather(key, value, -type, -day)) %>%
  
  rbind(sim_yn_map %>%
          filter(drift_threshold == 1) %>%
          ungroup() %>%
          mutate(hddm_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1))) %>%
          select(type, day, hddm_rt) %>%
          gather(key, value, -type, -day)) %>%
  
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1))),
         type = ifelse(type == 1, "HT", "RE"),
         key = factor(key, levels = c("true_rt", "grid_rt", "optim_rt", "hddm_rt")))

In [ ]:
sub_all_sim_data_long

First attempt. Not great.

In [ ]:
sub_all_sim_data %>%
  ggplot(aes(value, color = key, fill = key))+
  # geom_step(aes(y=..density..), stat="bin", binwidth=0.2)+
  geom_step(aes(y=..density..), stat="bin", bins = 30, linewidth = .75)+
  facet_grid(type ~ day)+
  scale_color_manual(values = c("gray", "#7570b3", "#e7298a", "#66a61e"))+
  scale_fill_manual(values = c("gray", NA, NA, NA))+
  theme(panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.position = "bottom")+
  labs(y = "", x = "", fill = "", color = "")+
ylim(0, 1.5)

In [ ]:
sub_all_sim_data_wide = sim_optim_best %>%
  select(true_rt, true_yesChosen, reactionTime, choice, type, day) %>%
  mutate(optim_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime),
         true_rt = ifelse(true_yesChosen == 0, (-1)*true_rt, true_rt)) %>%
  select(-true_yesChosen, -reactionTime, -choice) %>%
  gather(key, value, -type, -day, -true_rt) %>%

rbind(sim_grid_best %>%
          select(true_rt, true_yesChosen, reactionTime, choice, type, day) %>%
          mutate(grid_rt = ifelse(choice == "no", (-1)*reactionTime, reactionTime),
                 true_rt = ifelse(true_yesChosen == 0, (-1)*true_rt, true_rt)) %>%
          select(-true_yesChosen,-reactionTime, -choice) %>%
          gather(key, value, -type, -day, -true_rt)) %>%

rbind(sim_yn_map %>%
          filter(drift_threshold == 1) %>%
          ungroup() %>%
          mutate(hddm_rt = ifelse(pred_yesChosen == 1, pred_rt, pred_rt*(-1)),
                 true_rt = ifelse(yesChosen == 1, rt, rt*(-1))) %>%
          select(type, day, hddm_rt, true_rt) %>%
          gather(key, value, -type, -day, -true_rt )) %>%
  mutate(day = paste0("Day ", day),
         day = factor(day, levels = paste0("Day ", seq(1,11, 1))),
         # type = ifelse(type == 1, "HT", "RE"),
         key = factor(key, levels = c("grid_rt", "optim_rt", "hddm_rt")))


In [ ]:
sub_all_sim_data_wide %>%
  filter(type == 1) %>%
  ggplot()+
  geom_histogram(aes(x = true_rt, y=..density..), stat="bin", bins = 30, linewidth = .75, fill = "#1b9e77", alpha = .75)+
  geom_density(aes(x = value))+
  facet_grid(key ~ day)+
  theme(legend.position = "none",
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        panel.grid.minor = element_blank(),
        strip.text = element_text(size = 14))+
  labs(x = "", y = "", title = "Comparison of true and simulated for HT stimuli (sub-601)")

ggsave(file=paste0(fig_out_path, 'sub601_simDataComparisonHT1.jpg'), height = 9, width=20, units="in")
  

In [ ]:
sub_all_sim_data_wide %>%
  filter(type == 1) %>%
  ggplot()+
  geom_step(aes(x = true_rt, y=..density..), stat="bin", bins = 30, linewidth = .75, color = "#1b9e77")+
  geom_step(aes(x = value, y=..density..), stat="bin", bins = 30, linewidth = .75)+
  facet_grid(key ~ day)+
  theme(legend.position = "none",
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        panel.grid.minor = element_blank(),
        strip.text = element_text(size = 14))+
  labs(x = "", y = "", title = "Comparison of true and simulated for HT stimuli (sub-601)")

ggsave(file=paste0(fig_out_path, 'sub601_simDataComparisonHT2.jpg'), height = 9, width=20, units="in")
  

In [ ]:
sub_all_sim_data_wide %>%
  filter(type == 0) %>%
  ggplot()+
  geom_histogram(aes(x = true_rt, y=..density..), stat="bin", bins = 30, linewidth = .75, fill = "#d95f02", alpha = .75)+
  geom_density(aes(x = value))+
  facet_grid(key ~ day)+
  theme(legend.position = "none",
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        panel.grid.minor = element_blank(),
        strip.text = element_text(size = 14))+
  labs(x = "", y = "", title = "Comparison of true and simulated for RE stimuli (sub-601)")

ggsave(file=paste0(fig_out_path, 'sub601_simDataComparisonRE1.jpg'), height = 9, width=20, units="in")
  

In [ ]:
sub_all_sim_data_wide %>%
  filter(type == 0) %>%
  ggplot()+
  geom_step(aes(x = true_rt, y=..density..), stat="bin", bins = 30, linewidth = .75, color = "#d95f02")+
  geom_step(aes(x = value, y=..density..), stat="bin", bins = 30, linewidth = .75)+
  facet_grid(key ~ day)+
  theme(legend.position = "none",
        axis.ticks = element_blank(),
        axis.text = element_blank(),
        panel.grid.minor = element_blank(),
        strip.text = element_text(size = 14))+
  labs(x = "", y = "", title = "Comparison of true and simulated for RE stimuli (sub-601)")

ggsave(file=paste0(fig_out_path, 'sub601_simDataComparisonRE2.jpg'), height = 9, width=20, units="in")
  

# Other checks

Is the smaller negative log likelihoods for RE compared HT a thing for all other subjects too? Yes!

But wait. This isn't necessarily better fit. This isn't normalized. There's fewer RE trials so the sum of likelihood for each trial is smaller.

In [ ]:
all_optim_filtered %>%
  group_by(subnum, day, type) %>%
  filter(nll == min(nll)) %>%
  ggplot(aes(nll, fill = type))+
  geom_histogram(alpha = .5, bins = 30, position = "identity")+
  theme(legend.position = "bottom")+
  scale_fill_brewer(palette = "Dark2")+
  labs(fill = "")+
  facet_wrap(~subnum)

Where did the best optim estimates start from?

In [ ]:
all_optim_filtered %>%
  group_by(subnum, day, type) %>%
  filter(nll == min(nll))